In [1]:
import pandas as pd

from env import data_path

1. Attendance Data

* Load the attendance.csv file and calculate an attendnace percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

In [78]:
attendance_df = pd.read_csv(data_path + "/untidy-data/attendance.csv")
attendance_df

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [3]:
attendance_df.columns

Index(['Unnamed: 0', '2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
       '2018-01-05', '2018-01-06', '2018-01-07', '2018-01-08'],
      dtype='object')

In [15]:
attendance_df.shape

(4, 9)

In [4]:
attendance_df.rename(columns={"Unnamed: 0" : "Student"}, inplace=True)
attendance_df.head()

,Student,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [9]:
attendance_melt = attendance_df.melt(id_vars="Student", var_name="Date", value_name="Attendance")
attendance_melt.head()

,Student,Date,Attendance
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T


In [79]:
def convert_attendance(code):
    if code == "P":
        return 1
    elif code == "H":
        return 0.5
    elif code == "T":
        return .9
    else:
        return 0
    
attendance_melt['attendance_value'] = attendance_melt.Attendance.apply(convert_attendance)
attendance_melt.head()

,Student,Date,Attendance,attendance_value
0,Sally,2018-01-01,P,1.0
1,Jane,2018-01-01,A,0.0
2,Billy,2018-01-01,A,0.0
3,John,2018-01-01,P,1.0
4,Sally,2018-01-02,T,0.9


In [82]:
attendance_melt.groupby('Student').attendance_value.sum() / attendance_melt.Date.unique().size

Student
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: attendance_value, dtype: float64

In [141]:
# Alternative solution
alt_attendance = attendance_melt.replace({'P' : 1, 'A' : 0, 'H' : .5, 'T' : .9})
alt_attendance.groupby("Student").Attendance.sum() / 8

Student
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: Attendance, dtype: float64

2. Coffee Levels

* Read the coffee_levels.csv file.

In [61]:
coffee_df = pd.read_csv(data_path + "/untidy-data/coffee_levels.csv")
coffee_df.head()

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


* Transform the data so that each carafe is in it's own column.

In [62]:
coffee_df.coffee_carafe.unique()

array(['x', 'y', 'z'], dtype=object)

In [64]:
coffee_pivot = coffee_df.pivot(index='hour', columns='coffee_carafe', values='coffee_amount')
coffee_pivot.head()

coffee_carafe,x,y,z
hour,,,
8,0.816164,0.189297,0.999264
9,0.451018,0.521502,0.915990
10,0.843279,0.023163,0.144928
11,0.335533,0.235529,0.311495
12,0.898291,0.017009,0.771947


* Is this the best shape for the data?

In [65]:
coffee_df.shape, coffee_pivot.shape

((30, 3), (10, 3))

#### I think this is a better shape for the data because you can compare the coffee amounts between carafes at each hour by going across the row versus the first DF where you had to find the rows with the same hour, but different carafe to compare coffee amounts.

3. Cake Recipes

* Read the cake_recipes.csv data. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

In [53]:
cake_df = pd.read_csv(data_path + "/untidy-data/cake_recipes.csv")
cake_df.head()

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084


* Tidy the data as necessary.

In [83]:
cake_df.shape

(8, 5)

In [88]:
cake_split_df = cake_df['recipe:position'].str.split(":", expand=True)
cake_split_df.columns = ['recipe', 'position']
cake_split_df.head()

,recipe,position
0,a,bottom
1,a,top
2,b,bottom
3,b,top
4,c,bottom


In [142]:
tidy_cake_df = cake_df.drop(columns='recipe:position')
tidy_cake_df = pd.concat([cake_split_df, tidy_cake_df], axis=1)
tidy_cake_df.head()

,recipe,position,225,250,275,300
0,a,bottom,61.738655,53.912627,74.414730,98.786784
1,a,top,51.709751,52.009735,68.576858,50.228470
2,b,bottom,57.095320,61.904369,61.196980,99.248541
3,b,top,82.455004,95.224151,98.594881,58.169349
4,c,bottom,96.470207,52.001358,92.893227,65.473084


In [99]:
melt_tidy_cake_df = tidy_cake_df.melt(id_vars=['recipe', 'position'], var_name='temp', value_name='score')
melt_tidy_cake_df.head()

,recipe,position,temp,score
0,a,bottom,225,61.738655
1,a,top,225,51.709751
2,b,bottom,225,57.095320
3,b,top,225,82.455004
4,c,bottom,225,96.470207


* Which recipe, on average, is the best? recipe b

In [135]:
melt_tidy_cake_df.groupby('recipe').score.mean()

recipe
a    63.922201
b    76.736074
c    75.874748
d    62.864844
Name: score, dtype: float64

#### Recipe 'b' has an average score of 76.74 which is the highest

* Which oven temperature, on average, produces the best results? 275

In [103]:
melt_tidy_cake_df.groupby('temp').score.mean()

temp
225    71.306022
250    66.577437
275    74.886754
300    66.627655
Name: score, dtype: float64

#### A temperature of 275 produced the highest average score of 74.89

* Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees

In [132]:
best_score = melt_tidy_cake_df['score'].agg("max")
best_score

99.2485405378462

In [133]:
best_combo = melt_tidy_cake_df[melt_tidy_cake_df['score'] == best_score]
best_combo

,recipe,position,temp,score
26,b,bottom,300,99.248541


In [147]:
# Alternative solution
(melt_tidy_cake_df.groupby(['temp', 'recipe', 'position']).score.mean().idxmax(),
melt_tidy_cake_df.groupby(['temp', 'recipe', 'position']).score.mean().max())

(('300', 'b', 'bottom'), 99.2485405378462)

#### The best combo is recipe b, bottom rack, at 300 with a score of 99.25.